In [327]:
import os
import re
from datetime import datetime
import pandas as pd

In [328]:
with open(os.path.join("downloads/2024-11-06","rj_Praha_Brno_D03_2024-11-06_16-26-28.html"), "r", encoding="utf-8") as spojeni:
    spojeni = spojeni.read()

In [329]:
print("\n".join(spojeni.split(">")))

<html lang="cs" style=""
<head
<script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/destination?id=DC-12410592&amp;l=dataLayer&amp;cx=c"
</script
<script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-XRBBB4R33S&amp;l=dataLayer&amp;cx=c"
</script
<script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-982225530&amp;l=dataLayer&amp;cx=c"
</script
<script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-834094141&amp;l=dataLayer&amp;cx=c"
</script
<script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-819906300&amp;l=dataLayer&amp;cx=c"
</script
<script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-1018411943&amp;l=dataLayer&amp;cx=c"
</script
<script type="text/javascript" async="" charset="utf-8" src="https://www.gstatic.com/recaptcha/release

In [337]:
def oscrapuj_rj(slozka, soubor):
    with open(os.path.join(slozka,soubor), "r", encoding="utf-8") as spojeni:
        spojeni = spojeni.read()
    spoje = []
    spoj = None
    for radek in spojeni.split("</"):
        if len(radek) < 5000:
            radek = radek.replace("<sup>+1","")
            if re.search(r"\w{5,12},\s\d{1,2}\.\s\w{4,12}\s20\d\d", radek):
                datum = re.search(r"\w{5,12},\s\d{1,2}\.\s\w{4,12}\s20\d\d", radek).group(0)
                den = datum.split(",")[0]
                datum = datum.split(',')[1].replace("ledna","1.").replace("února","2.").replace("března","3.").replace("dubna","4.").replace("května","5.").replace("června","6.").replace('července','7.').replace('srpna','8.').replace('září','9.').replace('října','10.').replace('listopadu','11.').replace('prosince','12.').replace(' ','')
            if re.search(r"Spoj\s\d{2}:", radek):
                if spoj:
                    spoje.append(spoj)
                spoj = {}
                spoj['prodejce'] = "RJ"
                spoj['odkud'], spoj['kam'] = soubor.split('_')[1], soubor.split('_')[2]
                odjezdprijezd = radek.replace('+1','').split('>')[-1].split(' - ')
                oscrapovano = re.search(r'\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}', soubor).group(0)
                spoj['oscrapovano'] = datetime.strptime(oscrapovano, "%Y-%m-%d_%H-%M-%S")
                spoj['odjezd'] = datetime.strptime('-'.join(datum.split('-')[::-1]) + " " + odjezdprijezd[0].strip(), "%d.%m.%Y %H:%M")
                spoj['predstih'] = spoj['odjezd'] - spoj['oscrapovano']
                spoj['den'] = den
            if "Délka cesty" in radek:
                if spoj:
                    jizdni_doba = re.search(r"\d\d:\d\d",radek).group(0)
                    jizdni_doba = [int(x) for x in jizdni_doba.split(':')]
                    spoj['jizdni_doba'] = jizdni_doba[0] * 60 + jizdni_doba[1]
            if re.search(r"\d{1,5}&nbsp;Kč",radek):
                if spoj:
                    cena = re.search(r"\d{1,5}Kč", radek.replace(" ","").replace("&nbsp;","")).group(0)
                    spoj['cena'] = int(cena.split("Kč")[0])
                    if spoj['cena'] == 79:
                        print(radek)
            if "Volných míst" in radek:
                if spoj:
                    spoj['volnych_mist'] = int(re.search(r"\d{1,6}", radek).group(0))
            if "Vyprodáno" in radek:
                if spoj:
                    spoj['volnych_mist'] = 0
            if "přestup" in radek:
                prestupy = re.search(r'>\d',radek)
                if prestupy != None:
                    spoj['prestupy'] = int(prestupy.group(0)[1])
            if "Přímý" in radek:
                if spoj:
                    spoj['prestupy'] = 0
            if "M5 17.526c0 .788.341 1.494.875 1.987v1.145c0 .742.586 1.342 1.313 1.342.726 0 1.312-.6 1.312-1.342v-.448h7v.448c0 .734.586 1.342 1.313 1.342.717 0 1.312-.6 1.312-1.342v-1.145A2.698 2.698 0 0 0 19 17.526V8.58C19 5.447 15.867 5 12 5s-7 .447-7 3.579v8.947Zm3.063.895c-.727 0-1.313-.6-1.313-1.342 0-.743.586-1.342 1.313-1.342.726 0 1.312.6 1.312 1.342 0 .743-.586 1.342-1.313 1.342Zm7.874 0c-.726 0-1.312-.6-1.312-1.342 0-.743.586-1.342 1.313-1.342.726 0 1.312.6 1.312 1.342 0 .743-.586 1.342-1.313 1.342Zm1.313-5.368H6.75V8.579h10.5v4.474Z" in radek:
                if spoj:
                    spoj['prostredek'] = 'autobus'
            if "M7 2h10v1.5h-3.204l-.772 1.515C16.151 5.113 19 5.754 19 8.579v8.5c0 1.727-1.374 3.131-3.063 3.131l.998 1.03c.28.277.088.76-.306.76h-.954a.44.44 0 0 1-.306-.134L13.75 20.21h-3.5l-1.619 1.655a.406.406 0 0 1-.306.134h-.954c-.394 0-.586-.483-.315-.76l1.006-1.03C6.375 20.21 5 18.806 5 17.08v-8.5c0-2.834 2.866-3.47 6.004-3.565l.772-1.514H7V2Zm-.25 15.079c0 .743.586 1.342 1.313 1.342.726 0 1.312-.6 1.312-1.342 0-.743-.586-1.342-1.313-1.342-.726 0-1.312.6-1.312 1.342Zm0-4.921h4.375V8.579H6.75v3.579Zm7.875 4.92c0 .744.586 1.343 1.313 1.343.726 0 1.312-.6 1.312-1.342 0-.743-.586-1.342-1.313-1.342-.726 0-1.312.6-1.312 1.342Zm-1.75-4.92h4.375V8.579h-4.375v3.579Z" in radek:
                if spoj:
                    spoj['prostredek'] = 'vlak'

    return spoje

In [339]:
for x in os.listdir("downloads"):
    kam = "data" 
    os.makedirs(kam, exist_ok=True)
    den = []
    rj = [y for y in os.listdir(f"downloads/{x}") if y[0:3] == "rj_"] 
    print(f"{x}: {len(rj)}")
    for y in rj:
        print(y)
        den = den + oscrapuj_rj(f"downloads/{x}",y)
    nazev_souboru = "rj_" + x + ".parquet"
    if len(den) > 0:
        df_den = pd.DataFrame(den).sort_values(by="oscrapovano").reset_index(drop=True)
        df_den.to_parquet(os.path.join(kam,nazev_souboru))

2024-10-31: 0
2024-11-01: 0
2024-11-02: 0
2024-11-03: 0
2024-11-04: 0
2024-11-05: 0
2024-11-06: 312
rj_Brno_Praha_D00_2024-11-06_21-12-55.html
rj_Brno_Praha_D00_2024-11-06_21-43-15.html
rj_Brno_Praha_D01_2024-11-06_21-12-57.html
rj_Brno_Praha_D01_2024-11-06_21-43-18.html
rj_Brno_Praha_D02_2024-11-06_21-13-00.html
rj_Brno_Praha_D02_2024-11-06_21-43-20.html
rj_Brno_Praha_D03_2024-11-06_21-13-03.html
rj_Brno_Praha_D03_2024-11-06_21-43-22.html
rj_Brno_Praha_D04_2024-11-06_21-13-05.html
rj_Brno_Praha_D04_2024-11-06_21-43-25.html
rj_Brno_Praha_D05_2024-11-06_21-13-07.html
rj_Brno_Praha_D06_2024-11-06_21-13-10.html
rj_Brno_Praha_D07_2024-11-06_21-13-12.html
rj_Brno_Praha_D08_2024-11-06_21-13-14.html
rj_Brno_Praha_D09_2024-11-06_21-13-17.html
rj_Brno_Praha_D11_2024-11-06_21-43-29.html
rj_Brno_Praha_D12_2024-11-06_21-43-33.html
rj_Brno_Praha_D13_2024-11-06_21-43-34.html
rj_Brno_Praha_D14_2024-11-06_21-43-36.html
rj_Brno_Praha_D15_2024-11-06_21-43-39.html
rj_Brno_Praha_D16_2024-11-06_21-43-42.ht